In [1]:
# 環境設定與函式庫載入
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_palette("husl")

print("優化環境設定完成！")


優化環境設定完成！


In [ ]:
def load_and_prepare_data(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入並準備數據"""
    try:
        # 嘗試不同編碼載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                print(f"成功使用 {encoding} 編碼載入數據")
                break
            except:
                continue
        
        print(f"原始數據形狀: {df.shape}")
        print(f"欄位: {list(df.columns)}")
        
        # 處理日期時間
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位名稱
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤', '開']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高', '高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低', '低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤', '收']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量', '量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        print(f"欄位對應: {column_mapping}")
        
        # 轉換為日線數據
        daily_agg = {
            'Open': 'first',
            'High': 'max', 
            'Low': 'min',
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_agg = {k: v for k, v in daily_agg.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_agg).dropna()
        
        print(f"日線數據形狀: {daily_df.shape}")
        print(f"日期範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入錯誤: {e}")
        return None

# 載入數據
data = load_and_prepare_data()
if data is not None:
    print("\n前5筆數據:")
    print(data.head())
    
    # 繪製價格走勢
    if 'Close' in data.columns:
        plt.figure(figsize=(14, 6))
        plt.plot(data.index, data['Close'], linewidth=1, alpha=0.8)
        plt.title('TXF1 Daily Close Price')
        plt.xlabel('Date')
        plt.ylabel('Price')
        plt.grid(True, alpha=0.3)
        plt.show()


In [ ]:
class OptimizedMAStrategy:
    """優化的移動平均策略類別"""
    
    def __init__(self, fast_ma=20, slow_ma=60, stop_loss=0.02, take_profit=0.04, 
                 min_volume_ratio=1.0, transaction_cost=0.001):
        """
        初始化策略參數
        
        Parameters:
        - fast_ma: 短期移動平均週期
        - slow_ma: 長期移動平均週期  
        - stop_loss: 停損比例 (0.02 = 2%)
        - take_profit: 停利比例 (0.04 = 4%)
        - min_volume_ratio: 最小成交量倍數 (相對於20日均量)
        - transaction_cost: 交易成本比例 (0.001 = 0.1%)
        """
        self.fast_ma = fast_ma
        self.slow_ma = slow_ma
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.min_volume_ratio = min_volume_ratio
        self.transaction_cost = transaction_cost
    
    def calculate_indicators(self, df):
        """計算技術指標"""
        signals = df.copy()
        
        # 移動平均線
        signals['MA_Fast'] = signals['Close'].rolling(self.fast_ma).mean()
        signals['MA_Slow'] = signals['Close'].rolling(self.slow_ma).mean()
        
        # 成交量指標 (如果有成交量數據)
        if 'Volume' in signals.columns:
            signals['Volume_MA'] = signals['Volume'].rolling(20).mean()
            signals['Volume_Ratio'] = signals['Volume'] / signals['Volume_MA']
        else:
            signals['Volume_Ratio'] = 1.0
        
        # RSI 指標 (額外的過濾條件)
        delta = signals['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        signals['RSI'] = 100 - (100 / (1 + rs))
        
        return signals
    
    def generate_signals(self, df):
        """生成交易訊號"""
        signals = self.calculate_indicators(df)
        signals['Signal'] = 0
        
        # 基本條件：均線交叉
        golden_cross = (
            (signals['MA_Fast'] > signals['MA_Slow']) & 
            (signals['MA_Fast'].shift(1) <= signals['MA_Slow'].shift(1))
        )
        
        death_cross = (
            (signals['MA_Fast'] < signals['MA_Slow']) & 
            (signals['MA_Fast'].shift(1) >= signals['MA_Slow'].shift(1))
        )
        
        # 額外過濾條件
        volume_filter = signals['Volume_Ratio'] >= self.min_volume_ratio
        rsi_oversold = signals['RSI'] < 70  # 避免在超買時買入
        rsi_overbought = signals['RSI'] > 30  # 避免在超賣時賣出
        
        # 買入訊號：黃金交叉 + 成交量確認 + RSI不超買
        buy_signal = golden_cross & volume_filter & rsi_oversold
        
        # 賣出訊號：死亡交叉 + RSI不超賣
        sell_signal = death_cross & rsi_overbought
        
        signals.loc[buy_signal, 'Signal'] = 1
        signals.loc[sell_signal, 'Signal'] = -1
        
        return signals.dropna()
    
    def backtest(self, df, initial_capital=1000000):
        """執行回測"""
        signals = self.generate_signals(df)
        
        if len(signals) == 0:
            return signals, pd.DataFrame()
        
        # 初始化變數
        cash = initial_capital
        position = 0  # 持倉數量
        entry_price = 0
        portfolio_values = []
        trades = []
        
        for i, (date, row) in enumerate(signals.iterrows()):
            current_price = row['Close']
            signal = row['Signal']
            
            # 檢查停損停利 (如果有持倉)
            if position > 0:
                price_change = (current_price - entry_price) / entry_price
                
                # 停損
                if price_change <= -self.stop_loss:
                    pnl = (current_price - entry_price) * position
                    transaction_cost = abs(current_price * position) * self.transaction_cost
                    net_pnl = pnl - transaction_cost
                    cash += net_pnl
                    
                    trades.append({
                        'Date': date,
                        'Action': 'STOP_LOSS',
                        'Price': current_price,
                        'Entry_Price': entry_price,
                        'Position': position,
                        'PnL': net_pnl,
                        'Portfolio_Value': cash
                    })
                    
                    position = 0
                    entry_price = 0
                
                # 停利
                elif price_change >= self.take_profit:
                    pnl = (current_price - entry_price) * position
                    transaction_cost = abs(current_price * position) * self.transaction_cost
                    net_pnl = pnl - transaction_cost
                    cash += net_pnl
                    
                    trades.append({
                        'Date': date,
                        'Action': 'TAKE_PROFIT',
                        'Price': current_price,
                        'Entry_Price': entry_price,
                        'Position': position,
                        'PnL': net_pnl,
                        'Portfolio_Value': cash
                    })
                    
                    position = 0
                    entry_price = 0
            
            # 處理交易訊號
            if signal == 1 and position == 0:  # 買入訊號且無持倉
                position = 1  # 簡化為固定1單位
                entry_price = current_price
                transaction_cost = current_price * position * self.transaction_cost
                cash -= transaction_cost  # 扣除買入成本
                
                trades.append({
                    'Date': date,
                    'Action': 'BUY',
                    'Price': current_price,
                    'Entry_Price': entry_price,
                    'Position': position,
                    'PnL': -transaction_cost,
                    'Portfolio_Value': cash
                })
            
            elif signal == -1 and position > 0:  # 賣出訊號且有持倉
                pnl = (current_price - entry_price) * position
                transaction_cost = abs(current_price * position) * self.transaction_cost
                net_pnl = pnl - transaction_cost
                cash += net_pnl
                
                trades.append({
                    'Date': date,
                    'Action': 'SELL',
                    'Price': current_price,
                    'Entry_Price': entry_price,
                    'Position': position,
                    'PnL': net_pnl,
                    'Portfolio_Value': cash
                })
                
                position = 0
                entry_price = 0
            
            # 計算當前組合價值
            if position > 0:
                unrealized_pnl = (current_price - entry_price) * position
                current_portfolio_value = cash + unrealized_pnl
            else:
                current_portfolio_value = cash
            
            portfolio_values.append(current_portfolio_value)
        
        signals['Portfolio_Value'] = portfolio_values
        trades_df = pd.DataFrame(trades)
        
        return signals, trades_df

print("優化策略類別定義完成！")


In [ ]:
def calculate_performance_metrics(signals, trades_df, initial_capital=1000000):
    """計算績效指標"""
    if len(signals) == 0:
        return None
    
    # 基本回報指標
    final_value = signals['Portfolio_Value'].iloc[-1]
    total_return = (final_value / initial_capital) - 1
    
    # 時間相關計算
    trading_days = len(signals)
    years = trading_days / 252
    annualized_return = (final_value / initial_capital) ** (1/years) - 1 if years > 0 else 0
    
    # 風險指標
    daily_returns = signals['Portfolio_Value'].pct_change().dropna()
    
    if len(daily_returns) > 0 and daily_returns.std() > 0:
        sharpe_ratio = (daily_returns.mean() / daily_returns.std()) * np.sqrt(252)
        volatility = daily_returns.std() * np.sqrt(252)
    else:
        sharpe_ratio = 0
        volatility = 0
    
    # 最大回撤
    rolling_max = signals['Portfolio_Value'].cummax()
    drawdown = (signals['Portfolio_Value'] - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    
    # 交易相關指標
    if len(trades_df) > 0:
        completed_trades = trades_df[trades_df['Action'].isin(['SELL', 'STOP_LOSS', 'TAKE_PROFIT'])]
        if len(completed_trades) > 0:
            winning_trades = completed_trades[completed_trades['PnL'] > 0]
            win_rate = len(winning_trades) / len(completed_trades)
            avg_win = winning_trades['PnL'].mean() if len(winning_trades) > 0 else 0
            avg_loss = completed_trades[completed_trades['PnL'] < 0]['PnL'].mean() if len(completed_trades[completed_trades['PnL'] < 0]) > 0 else 0
            
            total_wins = winning_trades['PnL'].sum() if len(winning_trades) > 0 else 0
            total_losses = abs(completed_trades[completed_trades['PnL'] < 0]['PnL'].sum()) if len(completed_trades[completed_trades['PnL'] < 0]) > 0 else 0
            profit_factor = total_wins / total_losses if total_losses > 0 else float('inf')
        else:
            win_rate = avg_win = avg_loss = profit_factor = 0
        
        num_trades = len(trades_df)
    else:
        win_rate = avg_win = avg_loss = profit_factor = num_trades = 0
    
    return {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'volatility': volatility,
        'win_rate': win_rate,
        'profit_factor': profit_factor,
        'num_trades': num_trades,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'final_value': final_value
    }

def optimize_parameters(data, param_ranges, optimization_metric='sharpe_ratio', max_combinations=500):
    """參數最佳化函數"""
    if data is None or len(data) == 0:
        print("無可用數據進行最佳化")
        return None, None
    
    # 生成所有參數組合
    param_combinations = list(product(*param_ranges.values()))
    
    # 如果組合太多，隨機採樣
    if len(param_combinations) > max_combinations:
        import random
        param_combinations = random.sample(param_combinations, max_combinations)
        print(f"參數組合過多，隨機採樣 {max_combinations} 組合")
    
    print(f"開始測試 {len(param_combinations)} 種參數組合...")
    
    results = []
    param_names = list(param_ranges.keys())
    
    for i, params in enumerate(param_combinations):
        if i % 20 == 0:
            print(f"進度: {i+1}/{len(param_combinations)} ({(i+1)/len(param_combinations)*100:.1f}%)")
        
        try:
            # 建立策略參數字典
            strategy_params = dict(zip(param_names, params))
            
            # 建立策略實例
            strategy = OptimizedMAStrategy(**strategy_params)
            
            # 執行回測
            signals, trades = strategy.backtest(data)
            
            # 計算績效指標
            metrics = calculate_performance_metrics(signals, trades)
            
            if metrics is not None:
                # 合併參數和績效指標
                result = {**strategy_params, **metrics}
                results.append(result)
                
        except Exception as e:
            # 跳過有問題的參數組合
            continue
    
    if len(results) == 0:
        print("沒有成功的參數組合")
        return None, None
    
    results_df = pd.DataFrame(results)
    
    # 根據最佳化指標排序
    if optimization_metric in results_df.columns:
        if optimization_metric == 'max_drawdown':
            # 最大回撤越小越好（越接近0）
            best_idx = results_df['max_drawdown'].idxmax()
        else:
            # 其他指標越大越好
            best_idx = results_df[optimization_metric].idxmax()
        
        best_params = results_df.loc[best_idx]
    else:
        print(f"找不到最佳化指標: {optimization_metric}")
        best_params = results_df.iloc[0]
    
    print(f"最佳化完成！共測試 {len(results)} 個有效組合")
    
    return results_df, best_params

# 測試基礎策略
if data is not None:
    print("測試基礎策略...")
    basic_strategy = OptimizedMAStrategy()
    signals, trades = basic_strategy.backtest(data)
    
    if len(signals) > 0:
        metrics = calculate_performance_metrics(signals, trades)
        print("\\n=== 基礎策略績效 ===")
        for key, value in metrics.items():
            if isinstance(value, float):
                if 'return' in key or 'drawdown' in key or 'rate' in key:
                    print(f"{key}: {value:.2%}")
                else:
                    print(f"{key}: {value:.4f}")
            else:
                print(f"{key}: {value}")
    else:
        print("基礎策略無法生成有效訊號")


In [ ]:
# 定義參數搜索空間
param_ranges = {
    'fast_ma': [10, 15, 20, 25, 30],
    'slow_ma': [40, 50, 60, 70, 80],
    'stop_loss': [0.01, 0.02, 0.03, 0.04],
    'take_profit': [0.02, 0.04, 0.06, 0.08],
    'min_volume_ratio': [0.8, 1.0, 1.2],
    'transaction_cost': [0.0005, 0.001, 0.0015]
}

print("參數搜索空間:")
for param, values in param_ranges.items():
    print(f"  {param}: {values}")

total_combinations = 1
for values in param_ranges.values():
    total_combinations *= len(values)
print(f"\\n總參數組合數: {total_combinations:,}")

# 執行參數最佳化
if data is not None:
    optimization_results, best_params = optimize_parameters(
        data, 
        param_ranges, 
        optimization_metric='sharpe_ratio',
        max_combinations=300  # 限制最大測試數量
    )
    
    if optimization_results is not None:
        print("\\n=== 最佳參數組合 ===")
        print(f"最佳夏普比率: {best_params['sharpe_ratio']:.4f}")
        print("最佳參數:")
        for param in param_ranges.keys():
            print(f"  {param}: {best_params[param]}")
        
        print("\\n=== 最佳參數績效 ===")
        performance_keys = ['total_return', 'annualized_return', 'max_drawdown', 'win_rate', 'profit_factor', 'num_trades']
        for key in performance_keys:
            if key in best_params:
                value = best_params[key]
                if isinstance(value, float):
                    if 'return' in key or 'drawdown' in key or 'rate' in key:
                        print(f"  {key}: {value:.2%}")
                    else:
                        print(f"  {key}: {value:.4f}")
                else:
                    print(f"  {key}: {value}")
        
        print("\\n=== 前10名結果 ===")
        top_10 = optimization_results.nlargest(10, 'sharpe_ratio')
        display_cols = ['fast_ma', 'slow_ma', 'stop_loss', 'take_profit', 'sharpe_ratio', 'total_return', 'max_drawdown']
        print(top_10[display_cols].round(4))
    else:
        print("參數最佳化失敗")
else:
    print("無數據可進行最佳化")


In [ ]:
# 視覺化最佳化結果
if 'optimization_results' in locals() and optimization_results is not None:
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Parameter Optimization Results', fontsize=16)
    
    # 1. 風險回報散佈圖
    scatter = axes[0,0].scatter(
        optimization_results['total_return'], 
        optimization_results['sharpe_ratio'],
        c=optimization_results['max_drawdown'], 
        cmap='RdYlBu_r', 
        alpha=0.6,
        s=50
    )
    axes[0,0].set_xlabel('Total Return')
    axes[0,0].set_ylabel('Sharpe Ratio')
    axes[0,0].set_title('Risk-Return Profile')
    axes[0,0].grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=axes[0,0], label='Max Drawdown')
    
    # 標記最佳點
    axes[0,0].scatter(
        best_params['total_return'], 
        best_params['sharpe_ratio'],
        color='red', 
        s=100, 
        marker='*', 
        label='Best'
    )
    axes[0,0].legend()
    
    # 2. 移動平均參數熱力圖
    try:
        pivot_sharpe = optimization_results.pivot_table(
            values='sharpe_ratio', 
            index='slow_ma', 
            columns='fast_ma', 
            aggfunc='mean'
        )
        sns.heatmap(pivot_sharpe, annot=True, fmt='.3f', ax=axes[0,1], cmap='viridis')
        axes[0,1].set_title('MA Parameters vs Sharpe Ratio')
    except:
        axes[0,1].text(0.5, 0.5, 'Insufficient data for heatmap', ha='center', va='center')
        axes[0,1].set_title('MA Parameters Heatmap')
    
    # 3. 停損停利參數分析
    try:
        pivot_return = optimization_results.pivot_table(
            values='total_return', 
            index='stop_loss', 
            columns='take_profit', 
            aggfunc='mean'
        )
        sns.heatmap(pivot_return, annot=True, fmt='.3f', ax=axes[0,2], cmap='plasma')
        axes[0,2].set_title('Stop Loss/Take Profit vs Return')
    except:
        axes[0,2].text(0.5, 0.5, 'Insufficient data for heatmap', ha='center', va='center')
        axes[0,2].set_title('Stop Loss/Take Profit Analysis')
    
    # 4. 夏普比率分布
    axes[1,0].hist(optimization_results['sharpe_ratio'], bins=20, alpha=0.7, color='skyblue')
    axes[1,0].axvline(best_params['sharpe_ratio'], color='red', linestyle='--', linewidth=2, label='Best')
    axes[1,0].set_xlabel('Sharpe Ratio')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].set_title('Sharpe Ratio Distribution')
    axes[1,0].legend()
    axes[1,0].grid(True, alpha=0.3)
    
    # 5. 回報率 vs 最大回撤
    axes[1,1].scatter(
        optimization_results['max_drawdown'], 
        optimization_results['total_return'],
        alpha=0.6,
        c=optimization_results['sharpe_ratio'],
        cmap='viridis'
    )
    axes[1,1].scatter(
        best_params['max_drawdown'], 
        best_params['total_return'],
        color='red', 
        s=100, 
        marker='*', 
        label='Best'
    )
    axes[1,1].set_xlabel('Max Drawdown')
    axes[1,1].set_ylabel('Total Return')
    axes[1,1].set_title('Return vs Drawdown')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
    
    # 6. 交易次數 vs 勝率
    if 'win_rate' in optimization_results.columns and 'num_trades' in optimization_results.columns:
        scatter2 = axes[1,2].scatter(
            optimization_results['num_trades'], 
            optimization_results['win_rate'],
            alpha=0.6,
            c=optimization_results['profit_factor'],
            cmap='coolwarm'
        )
        axes[1,2].scatter(
            best_params['num_trades'], 
            best_params['win_rate'],
            color='red', 
            s=100, 
            marker='*', 
            label='Best'
        )
        axes[1,2].set_xlabel('Number of Trades')
        axes[1,2].set_ylabel('Win Rate')
        axes[1,2].set_title('Trades vs Win Rate')
        axes[1,2].legend()
        axes[1,2].grid(True, alpha=0.3)
        plt.colorbar(scatter2, ax=axes[1,2], label='Profit Factor')
    else:
        axes[1,2].text(0.5, 0.5, 'Trade data not available', ha='center', va='center')
        axes[1,2].set_title('Trading Statistics')
    
    plt.tight_layout()
    plt.show()
    
    # 參數重要性分析
    print("\\n=== 參數重要性分析 ===")
    param_importance = {}
    
    for param in param_ranges.keys():
        if param in optimization_results.columns:
            correlation = optimization_results[param].corr(optimization_results['sharpe_ratio'])
            param_importance[param] = abs(correlation)
    
    # 排序並顯示
    sorted_importance = sorted(param_importance.items(), key=lambda x: x[1], reverse=True)
    
    for param, importance in sorted_importance:
        print(f"  {param}: {importance:.4f}")
    
    # 繪製參數重要性
    if len(sorted_importance) > 0:
        params, importances = zip(*sorted_importance)
        
        plt.figure(figsize=(10, 6))
        bars = plt.bar(params, importances, alpha=0.7, color='lightcoral')
        plt.title('Parameter Importance (Correlation with Sharpe Ratio)')
        plt.ylabel('Absolute Correlation')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        
        # 添加數值標籤
        for bar, importance in zip(bars, importances):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                    f'{importance:.3f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()

else:
    print("無最佳化結果可視覺化")


In [ ]:
# 策略比較
if data is not None and 'best_params' in locals():
    
    # 定義不同版本的策略進行比較
    strategies = {
        'Basic MA (20/60)': OptimizedMAStrategy(
            fast_ma=20, slow_ma=60, 
            stop_loss=0, take_profit=0, 
            min_volume_ratio=1.0, transaction_cost=0
        ),
        'With Stop Loss/Profit': OptimizedMAStrategy(
            fast_ma=20, slow_ma=60, 
            stop_loss=0.02, take_profit=0.04, 
            min_volume_ratio=1.0, transaction_cost=0
        ),
        'With Transaction Cost': OptimizedMAStrategy(
            fast_ma=20, slow_ma=60, 
            stop_loss=0.02, take_profit=0.04, 
            min_volume_ratio=1.0, transaction_cost=0.001
        ),
        'Optimized Strategy': OptimizedMAStrategy(
            fast_ma=int(best_params['fast_ma']),
            slow_ma=int(best_params['slow_ma']),
            stop_loss=best_params['stop_loss'],
            take_profit=best_params['take_profit'],
            min_volume_ratio=best_params['min_volume_ratio'],
            transaction_cost=best_params['transaction_cost']
        )
    }
    
    # 執行所有策略的回測
    strategy_results = {}
    strategy_signals = {}
    
    print("執行策略比較...")
    
    for name, strategy in strategies.items():
        print(f"  測試策略: {name}")
        signals, trades = strategy.backtest(data)
        
        if len(signals) > 0:
            metrics = calculate_performance_metrics(signals, trades)
            strategy_results[name] = metrics
            strategy_signals[name] = signals
        else:
            print(f"    {name} 策略無法生成有效訊號")
    
    # 比較結果表格
    if strategy_results:
        comparison_df = pd.DataFrame(strategy_results).T
        
        print("\\n=== 策略績效比較 ===")
        display_metrics = ['total_return', 'annualized_return', 'sharpe_ratio', 'max_drawdown', 'win_rate', 'num_trades']
        
        for metric in display_metrics:
            if metric in comparison_df.columns:
                print(f"\\n{metric.upper()}:")
                for strategy_name in comparison_df.index:
                    value = comparison_df.loc[strategy_name, metric]
                    if isinstance(value, float):
                        if 'return' in metric or 'drawdown' in metric or 'rate' in metric:
                            print(f"  {strategy_name:<25}: {value:>8.2%}")
                        else:
                            print(f"  {strategy_name:<25}: {value:>8.4f}")
                    else:
                        print(f"  {strategy_name:<25}: {value:>8}")
        
        # 繪製策略比較圖
        plt.figure(figsize=(16, 12))
        
        # 1. 淨值曲線比較
        plt.subplot(2, 2, 1)
        for name, signals in strategy_signals.items():
            if len(signals) > 0:
                normalized_value = signals['Portfolio_Value'] / signals['Portfolio_Value'].iloc[0]
                plt.plot(signals.index, normalized_value, label=name, linewidth=2)
        
        plt.title('Normalized Portfolio Value Comparison')
        plt.xlabel('Date')
        plt.ylabel('Normalized Value')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        # 2. 回報率比較
        plt.subplot(2, 2, 2)
        returns = [strategy_results[name]['total_return'] for name in strategy_results.keys()]
        colors = ['skyblue', 'lightgreen', 'lightcoral', 'gold']
        bars = plt.bar(range(len(returns)), returns, color=colors[:len(returns)])
        plt.title('Total Return Comparison')
        plt.ylabel('Total Return')
        plt.xticks(range(len(returns)), list(strategy_results.keys()), rotation=45)
        
        # 添加數值標籤
        for bar, ret in zip(bars, returns):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{ret:.1%}', ha='center', va='bottom')
        
        # 3. 夏普比率比較
        plt.subplot(2, 2, 3)
        sharpe_ratios = [strategy_results[name]['sharpe_ratio'] for name in strategy_results.keys()]
        bars = plt.bar(range(len(sharpe_ratios)), sharpe_ratios, color=colors[:len(sharpe_ratios)])
        plt.title('Sharpe Ratio Comparison')
        plt.ylabel('Sharpe Ratio')
        plt.xticks(range(len(sharpe_ratios)), list(strategy_results.keys()), rotation=45)
        
        for bar, sharpe in zip(bars, sharpe_ratios):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                    f'{sharpe:.3f}', ha='center', va='bottom')
        
        # 4. 最大回撤比較
        plt.subplot(2, 2, 4)
        drawdowns = [strategy_results[name]['max_drawdown'] for name in strategy_results.keys()]
        bars = plt.bar(range(len(drawdowns)), drawdowns, color=colors[:len(drawdowns)])
        plt.title('Maximum Drawdown Comparison')
        plt.ylabel('Max Drawdown')
        plt.xticks(range(len(drawdowns)), list(strategy_results.keys()), rotation=45)
        
        for bar, dd in zip(bars, drawdowns):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() - 0.005, 
                    f'{dd:.1%}', ha='center', va='top', color='white', fontweight='bold')
        
        plt.tight_layout()
        plt.show()
        
        # Buy & Hold 基準比較
        if 'Close' in data.columns:
            buy_hold_return = (data['Close'].iloc[-1] / data['Close'].iloc[0]) - 1
            print(f"\\n=== 基準比較 ===")
            print(f"Buy & Hold Return: {buy_hold_return:.2%}")
            
            print("\\n策略 vs Buy & Hold 超額回報:")
            for name, results in strategy_results.items():
                excess_return = results['total_return'] - buy_hold_return
                print(f"  {name:<25}: {excess_return:>8.2%}")
    
    else:
        print("無策略結果可比較")

else:
    print("無數據或最佳參數可進行策略比較")
